# Reglas de asociación

Este cuaderno tiene como propósito ayudarte a realizar una búsqueda de reglas de asociación en tus datos, las cuales describen relaciones entre las variables en estos. Por ejemplo, si tus datos fueran recibos de compras, las reglas de asociación señalarían que productos se compran juntos con cierta frecuencia; si tus datos fueran libros o películas comprados o rentados por los clientes de una biblioteca o un servicio de películas como Netflix, entonces los reglas de asociación indicarían que libros o películas son leídos o vistos frecuentemente por una misma persona.

## Biblioteca

A lo largo del desarrollo de la Inteligencia Artificial y el Aprendizaje Automático se han utilizado una variedad de lenguajes de programación, incluyendo lenguajes diseñados específicamente para esas áreas, como es el caso de Lisp y Prolog. Actualmente, con la prevalencial aprendizaje automático subsimbólico —esto es, basado más en cálculos numéricos que en lógica y procesamiento simbólico— uno de los lenguajes más usados en el área es Python, para el cual se han estado desarrollando una amplia colección de bibliotecas que facilitan mucho el desarrollo y aplicación del aprendizaje automático a una variedad de problemas.

Proceceremos así a cargar la biblioteca Pandas para la gestión de datos, así como la biblioteca Apyori para la búsqueda de reglas de asociación, usando el algoritmo conocido como [Apriori](https://en.wikipedia.org/wiki/Apriori_algorithm).

Como esta biblioteca no se usan con tanta frecuencia, necesitamos instalarla antes de cargarla.


In [ ]:
import pandas as pd
!pip install apyori
from apyori import apriori

Además, como estás ejecutando este código de manera remota, en Google Colab, necesitamos también incluir bibliotecas para subir tu archivo CSV.

In [ ]:
import io
from google.colab import files

## Preparación de datos

Procedemos entonces a subir los datos en tres etapas: damos el nombre del archivo de datos a subir, lo seleccionamos y luego lo cargamos en un *dataframe* (una especie de hoja de cálculo).

Confirmamos la lectura completa del archivo visualizando sus columnas (variables) y los tipos de éstas.

In [ ]:
filename = input('Nombre de tu archivo CSV: ')
print(f'Confirmo el nombre del archivo: {filename}')

uploaded = files.upload()
dataframe = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='utf-8')
dataframe.dtypes

Ahora seleccionamos las variables que se usarán para realizar la búsqueda de reglas de asociación.

La búsqueda de reglas de asociación funciona mejor cuando tenemos datos de tipo nominal u ordinal. Esto es, con columnas cuyos valores son cadenas de caracteres (apareceran arriba como de tipo `object`.

In [ ]:
# La lista con los nombres de todas las variables.
all_variables = list(dataframe.columns)

# Lee los nombres de las variables que serán usadas para realizar
# el análisis de cúmulos.
print('¿Cuáles variables te interesa usar para el análisis?')
print('Escribe sus nombres, uno en cada renglón.')
print('Cuando hayas acabado, teclea solamente ↲')
clustering_variables = []
i = 1

variable = input(f'Variable {i}: ')
while variable != '':
  if not (variable in all_variables):
    print(f'La variable {variable} no existe en tus datos. Intenta de nuevo')
  else:
    clustering_variables.append(variable)
    i += 1
  variable = input(f'Variable {i}: ')

print(f'¡Listo! Te confirmo las variables que escogiste:\n{clustering_variables}')

A continuación separamos las variables que se van a usar para la búsqueda de reglas de asociación.

In [ ]:
data = dataframe[clustering_variables]

## Búsqueda de reglas de asociación

Una regla de asociación tiene la forma general
$$
  x_1, x_2,\dots, x_n \to y
$$
y se interpreta como ‘si $x_1,x_2,\dots, x_n$ aparecen en un renglón, entonces es *frecuente* que $y$ también aparezca en ese renglón‘.

El ejemplo clásico es $n=2$, $x_1 = \mbox{hombre}$, $x_2 = \mbox{cerveza}$ y $y = \mbox{pañales}$, que se lee como ‘un hombre que compra cervezas (en el supermercado) con frecuencia compra también pañales’ ---lo que llevó a poner los pañales cerca de las cervezas en el supermercado.

Otro ejemplo sería, ‘quién ve la película $x$, con frecuencia ve la película $y$’.

### Preparación final de datos.

Para la búsqueda de reglas de asociación, primero convertimos la “hoja de cálculo” que tenemos en una lista de renglones, que es el formato solicitado por la biblioteca que estamos utilizando.

In [ ]:
records = []
for i in range(1, data.shape[0]):
    records.append([str(data.values[i, j]) for j in range(data.shape[1])])

# Imprime el primero y el último renglones.
print(records[0])
print(records[len(records)-1])

### Ejecución del algoritmo *Apriori*

El algoritmo de búsqueda de reglas de asociación recibe cuatro argumentos además de la lista de renglones:

1. *Soporte mínimo* (`min_support`): el porcentaje mínimo de renglones en los que debe aparecer un elemento para poder incluirlo en una regla de asociación.

2. *Confianza mínima* (`min_confidence`): el porcentaje de veces que $y$ tiene que aparecer cuando aparece $x$ para poder incluir la regla $x\to y$.

3. *Impulso mínimo* (`min_lift`): el valor mínimo del cociente de la confianza en $x\to y$ entre el soporte para $y$. Esto es, el porcentaje mínimo de las veces que aparece $y$ cuando aparece $x$ entre el porcentaje de veces que aparece $y$. Se puede interpretar como la confianza en la regla de parte de $y$.

Valores relativamente grandes en estos argumentos tienden a producir pocas reglas, con buen sustento en los datos, en tanto que valores relativamente pequeños tienden a producir muchas reglas pero con menor sustento en los datos. 


In [ ]:
support = 0.2
confidence = 0.4
lift = 1.0

Ejecutamos el algoritmo y recuperamos las reglas encontradas, vemos cuántas son y luego las ponemos en forma de hoja de cálculo para facilitar su visualización y almacenado.

In [ ]:
rules = apriori(records, min_support = support, 
                min_confidence = confidence, min_lift = lift)
results = list(rules)
print(f'Se encontraron {len(results)} reglas.')
results_frame = pd.DataFrame(results)

Finalmente, mostramos las reglas encontradas.

In [ ]:
results_frame.head(len(results_frame)-1)